In [1]:
from telegram import Update, InputFile
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ConversationHandler, ContextTypes
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
FROM, TO, DATE, CLASS, TRAIN_TYPE, NUM_PASSENGERS, PASSENGER_DETAILS, CREDENTIALS, CAPTCHA = range(9)


In [3]:
user_sessions = {}

In [4]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Welcome to the IRCTC Ticket Bot! Let's start your booking.\nFrom which station are you traveling?")
    return FROM

In [5]:
async def from_station(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_sessions[update.effective_user.id] = {'from': update.message.text}
    await update.message.reply_text("To which station are you going?")
    return TO

In [6]:
async def to_station(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_sessions[update.effective_user.id]['to'] = update.message.text
    await update.message.reply_text("Enter date of journey (dd-mm-yyyy):")
    return DATE

In [9]:
async def date_of_journey(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_sessions[update.effective_user.id]['date'] = update.message.text
    await update.message.reply_text("Which class? (SL, 3AC, 2AC, 2S, etc):")
    return CLASS

async def travel_class(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_sessions[update.effective_user.id]['class'] = update.message.text
    await update.message.reply_text("Train type or train number?")
    return TRAIN_TYPE

async def train_type(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_sessions[update.effective_user.id]['train_type'] = update.message.text
    await update.message.reply_text("How many passengers?")
    return NUM_PASSENGERS

async def num_passengers(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_sessions[update.effective_user.id]['num_passengers'] = int(update.message.text)
    await update.message.reply_text("Enter passenger details in format: Name,Age,Gender (one per line)")
    return PASSENGER_DETAILS

async def passenger_details(update: Update, context: ContextTypes.DEFAULT_TYPE):
    passengers = update.message.text.split('\n')
    user_sessions[update.effective_user.id]['passengers'] = [p.split(',') for p in passengers]
    await update.message.reply_text("Enter your IRCTC username and password (comma separated):")
    return CREDENTIALS

async def credentials(update: Update, context: ContextTypes.DEFAULT_TYPE):
    creds = update.message.text.split(',')
    user_sessions[update.effective_user.id]['credentials'] = {'username': creds[0], 'password': creds[1]}
    await update.message.reply_text("Thanks! We're starting the booking process now.")

    # from automation_script import automate_booking
    # captcha_image_path = automate_booking(user_sessions[update.effective_user.id])

    # Send captcha to user
    # await update.message.reply_photo(InputFile(captcha_image_path), caption="Enter the captcha shown:")
    return CAPTCHA

In [ ]:
from automation_script import automate_booking
captcha_image_path = automate_booking(user_sessions[update.effective_user.id])


In [ ]:
async def handle_captcha(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_sessions[update.effective_user.id]['captcha'] = update.message.text
    # from automation_script import complete_booking
    # ticket_pdf_path = complete_booking(user_sessions[update.effective_user.id])
    # await update.message.reply_document(InputFile(ticket_pdf_path), caption="Here is your booked ticket!")
    return ConversationHandler.END